In [131]:
from numpy import sqrt
from numpy import array, dot
from os    import makedirs, path
from inspect import getargvalues, currentframe, stack

from ase import Atoms
from ase.build import bulk, fcc111, surface
from ase.io import write
from ase.visualize import view


In [132]:
class GenerateStrectchFilm:
    def __init__(self, dir_path='./y_dir', name = 'Generate Strectch Film for hcp or fcc'):
        self.dir_path = dir_path
        self.name = name

    def generate_film(self, symbol = None, structure = None, num_layers = None, replic_z = None, my_vacuum = None, slice_plane = (0,0,1), my_tol=1e-6, my_periodic = True,
                      a_fcc = 2.95*sqrt(2.0),
                      a_hcp = 2.95 , my_covera = sqrt(8.0/3.0),
                      if_write = False, myfile_path = None, myformat= 'vasp', mydirect = True, myvasp5 = True):
        # parameters : 1st line: general setting
        #              2nd 3rd line: fcc, hcp parameters
        # when we use surface() function, my_bulk must be a conventional cell, not a primitive cell, so set cubic=True
        calling_function = stack()[1].function
        if structure == 'fcc':
            my_bulk = bulk(symbol, structure, a=a_fcc, cubic=True)
        elif structure == 'hcp':
            my_bulk = bulk(symbol, structure, a = a_hcp, covera = my_covera, cubic=True)
        else:
            print_after_what('structure', structure, calling_function, after_what= 'is not supported')

        if num_layers:
            num_rep_z = int(num_layers/len(my_bulk.numbers))
        elif replic_z:
            num_rep_z = replic_z
        else:
            print_after_what('num_layers and replic_z', None, calling_function, after_what= 'none')

        my_slab = surface(my_bulk, slice_plane , num_rep_z, vacuum = my_vacuum, tol=my_tol, periodic=my_periodic)
        my_slab.wrap()
        if if_write:
            write(myfile_path, my_slab, format=myformat, direct=mydirect, vasp5=myvasp5)
        return my_slab



def check_input(args = [], check_type = 'none'):
    # check_input_blank function is not correct, I think it's useless
    # first line  - check_input_none
    # second line - check_input_blank

    def check_input_none(args = [], calling_function = ''):
        # print(calling_function)
        for arg_name, arg_value in args.items():
            if arg_value is None:
                raise ValueError(f"In '{calling_function}': Argument '{arg_name}' is missing or None")
        
    # Get the name of the calling function
    calling_function = stack()[1].function
    if check_type == 'none':
        check_input_none(args, calling_function )
    else:
        print_after_what('check_type', calling_function, after_what= 'is not supported')

def print_after_what(char_name = '', variable = None,calling_function = '', specified_blank = '', character = '||', after_what = 'none'):
    # print(f"the value is None! - {calling_function}")
    print(f"the {character} {char_name} {variable} {character} is {after_what}! - {calling_function}")
    return specified_blank

# move atoms
def move_atoms(self, translate_matrix = array([0.1, 0.1, 0.0])):
    my_cell = array(self.get_cell())
    self.positions[:,:] += dot(translate_matrix, my_cell)
    self.wrap()
    print('complete move atoms')

# stretch unit cell
def uni_stretch(self, stretch_factor = [0.997, 0.998, 0.999, 1.000, 1.001, 1.002, 1.003], 
                base_filename = 'POSCAR_uniaxial', special_filename = '', my_path = './dump/'):
    self_copy = self.copy()
    my_cell = array(self_copy.get_cell())
    for i in stretch_factor:
        temp = my_cell.copy()
        temp[:2,0] = my_cell[:2,0] * i
        self_copy.set_cell(temp, scale_atoms=True)
        formatted_i = f'{i:.3f}'
        filename = f'{my_path}{base_filename}_{special_filename}_{formatted_i}'
        makedirs(path.dirname(filename), exist_ok=True)
        write(filename, self_copy, format='vasp', direct=True, vasp5=True)
    print('complete uniaxial stretch')

def bi_stretch(self, stretch_factor = [0.997, 0.998, 0.999, 1.000, 1.001, 1.002, 1.003],
               base_filename = 'POSCAR_biaxial', special_filename = '', tag_split = '_', my_path = './dump/'):
    self_copy = self.copy()
    my_cell = array(self_copy.get_cell())
    for i in stretch_factor:
        temp = my_cell.copy()
        temp[:2,:2] = my_cell[:2,:2] * i
        self_copy.set_cell(temp, scale_atoms=True)
        formatted_i = f'{i:.3f}'
        filename = f'{my_path}' + f'{base_filename}{tag_split}{special_filename}{tag_split}' + f'{formatted_i}'
        makedirs(path.dirname(filename), exist_ok=True)
        write(filename, self_copy, format='vasp', direct=True, vasp5=True)
    print('complete biaxial stretch')


#def mywrite







In [133]:
# generate_strectch_film = GenerateStrectchFilm()
# print(generate_strectch_film.__dict__)
# generate_strectch_film.generate_film(symbol = 'Au', structure = 'fcc', num_layers = 3, my_vacuum = 20, slice_plane = (1,0,0), a_fcc = 2.95*sqrt(2.0))
